# BERT's playground 
Hello there! Welcome on BERT's playground. You may play with BERT here and see what he can do but always make sure he feels respected and admired.

## Setups


In [1]:
import pickle
import numpy as np
import tensorflow as tf
import masking
import BERT

from Vectorisation import Vectorisation
from Config import Config
from MaskedLanguageModel import MaskedLanguageModel
from MaskedTextGenerator import MaskedTextGenerator

with open("./ml4science_data.pkl", "rb") as fp:
    data_dict = pickle.load(fp)

config = Config()
vec = Vectorisation(config=config)

In [2]:
# Prepare data for masked language model
encoded = vec.encode_dict(data_dict)
x_masked_encoded, y_masked_encoded, sample_weights = masking.mask_input_and_labels(encoded, config.TOKEN_DICT)
print(x_masked_encoded.shape, y_masked_encoded.shape, sample_weights.shape)

mlm_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded, sample_weights))
mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE)

print(mlm_ds)

(254, 256) (254, 256) (254, 256)
<_BatchDataset element_spec=(TensorSpec(shape=(None, 256), dtype=tf.int32, name=None), TensorSpec(shape=(None, 256), dtype=tf.int32, name=None), TensorSpec(shape=(None, 256), dtype=tf.float64, name=None))>


In [6]:
# TODO: continue implementing everything downstream so that BERT can finally run freely on his playground just as he wishes
sample_tokens = vec.encode(data_dict['sequences'][0]['sequence'])
print(sample_tokens[0])

[26  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [4]:
generator_callback = MaskedTextGenerator(sample_tokens)

bert_masked_model = BERT.create_masked_language_bert_model()
bert_masked_model.summary()

bert_masked_model.fit(mlm_ds, epochs=5, callbacks=[generator_callback])
bert_masked_model.save("bert_mlm_imdb.h5")

TypeError: MaskedTextGenerator.__init__() missing 2 required positional arguments: 'mask_token_id' and 'mapping_dict'